In [1]:
cd /u/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/preprocess/

/v/filer4b/v20q001/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/preprocess


In [2]:
import annotation_stats as db
import re

labeled_comment_ids = db.get_labeled_thrice_comments()
conservative_comment_ids = list(set([c_id for c_id in 
        db.get_all_comments_from_subreddit("Conservative") if c_id in labeled_comment_ids]))
#conservative_sentence_ids = get_sentence_ids_for_comments(conservative_comment_ids)
collapse_f = lambda lbl_set: 1 if lbl_set.count(1) >=2 else -1
conserv_ids, conserv_texts, conserv_lbls = db.get_texts_and_labels_for_sentences(conservative_comment_ids, add_punctuation_features_to_text=False, collapse=collapse_f)
sentiments = db.get_sentiments(conserv_ids)

emoticon_RE_str = '(?::|;|=)(?:-)?(?:\)|\(|D|P)'
question_mark_RE_str = '\?'
exclamation_point_RE_str = '\!'
# Any combination of multiple exclamation points and question marks
interrobang_RE_str = '[\?\!]{2,}'
puncts = [''] * len(conserv_texts)

for i, sentence in enumerate(conserv_texts):
    sentence = sentence.encode('utf-8')
    if len(re.findall(r'%s' % emoticon_RE_str, sentence)) > 0:
        puncts[i] += " PUNCxEMOTICON"
    if len(re.findall(r'%s' % exclamation_point_RE_str, sentence)) > 0:
        puncts[i] += " PUNCxEXCLAMATION_POINT"
    if len(re.findall(r'%s' % question_mark_RE_str, sentence)) > 0:
        puncts[i] += " PUNCxQUESTION_MARK"
    if len(re.findall(r'%s' % interrobang_RE_str, sentence)) > 0:
        puncts[i] += " PUNCxINTERROBANG"

    if any([len(s) > 2 and str.isupper(s) for s in sentence.split(" ")]):
        puncts[i] += " PUNCxUPPERCASE" 
    
    puncts[i] = puncts[i].strip()

PWD!!!!! /v/filer4b/v20q001/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/preprocess
train/dev database path: /u/npockrus/NLP/finalProject/venv/src/irony-classifier/data/ironate-dk3.sqlite
test database path: /u/npockrus/NLP/finalProject/venv/src/irony-classifier/data/ironate-dk3.sqlite
5019 comments have been labeled by >= 3 people


/u/npockrus/NLP/finalProject/venv/src/irony-classifier/lib/annotation_stats.py:64: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  config.read("irony.ini")


## Store labels and sentiment only 

In [3]:
cd /u/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/preprocess/

/v/filer4b/v20q001/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/preprocess


In [4]:
import pickle
with open('text-sentiment-label.p', 'wb') as f:
    pickle.dump({ sentence:{'label': lbl, 'sentiment': sentiment} for sentence, sentiment, lbl in zip(conserv_texts, sentiments, conserv_lbls) }, f)

## Store all features

In [7]:
author, subreddits = [], []
for id_ in conserv_ids:
    author.append(db.get_user_ids([db._get_comment_id_for_sentence(id_)]))
    subreddits.append(db.get_all_previous_subreddits_for_user(author[-1][0][0].decode('utf-8')))

In [8]:
cd /u/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/features/allFeatures/

/v/filer4b/v20q001/npockrus/NLP/finalProject/venv/src/irony-classifier/data/conservative/features/allFeatures


In [9]:
import pickle
with open('features.p', 'wb') as f:
    pickle.dump({ sentence:{'label': label, 'subreddits': subreddit, 'sentiment': sentiment, 'punctuation':punct} for sentence, label, subreddit, sentiment, punct in zip(conserv_texts, conserv_lbls, subreddits, sentiments, puncts) }, f)

In [10]:
print len(subreddits)
print subreddits[0]
print author[0]
print conservative_comment_ids[0]

949
[u'politics', u'pics', u'canada', u'worldnews', u'news', u'AskReddit', u'WTF', u'cordcutters', u'HistoryPorn', u'Calgary', u'occupywallstreet', u'Frugal', u'videos', u'worldpolitics', u'JusticePorn', u'atheism', u'funny', u'TrueAtheism', u'minimalism', u'technology', u'Quebec', u'dataisbeautiful', u'economy', u'MMA', u'ThriftStoreHauls', u'aww', u'personalfinance', u'nottheonion', u'nononono', u'iphone', u'Military', u'LearnUselessTalents', u'TheFutureIsNow', u'Charlotte', u'Justrolledintotheshop', u'electricvehicles', u'ChevyVolt', u'conspiracy', u'Scams', u'USNEWS']
[(u'wazzel2u',)]
3
